In [1]:
# Import standard library modules
import sys

# Set the relative path to the project root directory
relative_path_to_root = "../../"

# Add the project root to the system path for importing in-house modules
sys.path.append(relative_path_to_root)

# Import in-house modules from the 'utilities' package
from utilities import load_data, save_data
from utilities import save_data
from utilities import split_dataset_by_date

In [2]:
# Data manipulation and analysis
import pandas as pd

In [3]:
file_name = "updated_w_nas"
file_path = f"../../data/raw_data/{file_name}"

raw_data = load_data(file_path)

╔═══════════════════════════════════════════════════════════════╗
║ File `updated_w_nas.csv.bz2` loaded from `updated_w_nas.zip`  ║
╚═══════════════════════════════════════════════════════════════╝


In [4]:
print("Shape:", raw_data.shape)
raw_data.head()

Shape: (2121735, 18)


,Date,Ticker,Adjusted Close,Today to Tomorrow,Yesterday to Today,Next Day Close,Previous Day Close,Return,Volatility,RSI,SMA_50,SMA_100,SMA_200,Upper Band,Lower Band,Support,Resistance,Action
0,2008-01-02,A,23.256388,-1.0,-1.0,23.025745,23.538284,-0.011976,0.015704,48.827595,23.314173,23.299887,23.564934,24.727250,22.540232,21.392027,24.351938,short
1,2008-01-02,AAPL,5.876342,1.0,-1.0,5.879056,5.974059,-0.016357,0.018937,59.067417,5.518483,4.939064,4.197630,6.135834,5.403559,4.637375,6.026838,buy
2,2008-01-02,ABBV,0.000000,0.0,0.0,0.000000,0.000000,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
3,2008-01-02,ABNB,0.000000,0.0,0.0,0.000000,0.000000,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
4,2008-01-02,ABT,18.130213,-1.0,-1.0,18.019760,18.240650,-0.006054,0.010484,34.677270,18.138458,17.628250,17.709028,19.233109,18.221802,16.775555,19.134012,short


In [5]:
todays_date = "2024-10-28"

historical_data, todays_data = split_dataset_by_date(raw_data, todays_date)

print("Todays Date:", todays_date)

Todays Date: 2024-10-28


In [6]:
print("Shape:", todays_data.shape)
todays_data.head()

Shape: (501, 18)


,Date,Ticker,Adjusted Close,Today to Tomorrow,Yesterday to Today,Next Day Close,Previous Day Close,Return,Volatility,RSI,SMA_50,SMA_100,SMA_200,Upper Band,Lower Band,Support,Resistance,Action
0,2024-10-28,A,131.62,NaN,1.0,NaN,130.19,0.010984,0.012264,21.425411,140.169080,136.64084,137.961560,151.502270,128.73973,130.1900,148.24400,NaN
1,2024-10-28,AAPL,233.49,NaN,1.0,NaN,231.41,0.008988,0.013784,63.955185,226.950400,222.17525,201.011900,237.876570,222.29343,216.3200,236.48000,NaN
2,2024-10-28,ABBV,189.74,NaN,1.0,NaN,187.85,0.010061,0.008103,45.739376,192.653440,183.30086,174.407040,196.414630,185.96626,186.5400,197.77069,NaN
3,2024-10-28,ABNB,135.82,NaN,1.0,NaN,134.58,0.009214,0.014882,57.457336,125.337400,133.32010,143.597980,140.080760,125.49424,114.2800,137.19000,NaN
4,2024-10-28,ABT,114.13,NaN,-1.0,NaN,114.22,-0.000788,0.010339,50.131912,114.222595,109.51705,109.521515,119.484985,111.09392,110.2504,119.39000,NaN


In [7]:
def prepare_data_v2(main_data: pd.DataFrame) -> pd.DataFrame:
    # Create a copy of the input DataFrame
    df = main_data.copy().reset_index(drop=True)
    
    # Convert the `Date` column to datetime
    df["Date"] = pd.to_datetime(df["Date"])
    
    # Extract year, month, and day from the `Date` column
    df["Year"] = df["Date"].dt.year
    df["Month"] = df["Date"].dt.month
    df["Day"] = df["Date"].dt.day
    
    # Set the index to `Date` and `Ticker`
    df = df.set_index(["Date", "Ticker"])
    
    return df

In [8]:
predict_data_raw = todays_data.copy()

predict_data_raw = prepare_data_v2(predict_data_raw)

print("Shape:", predict_data_raw.shape)
predict_data_raw

Shape: (501, 19)


Adjusted Close  Today to Tomorrow  Yesterday to Today  \
Date       Ticker                                                          
2024-10-28 A              131.620                NaN                 1.0   
           AAPL           233.490                NaN                 1.0   
           ABBV           189.740                NaN                 1.0   
           ABNB           135.820                NaN                 1.0   
           ABT            114.130                NaN                -1.0   
...                           ...                ...                 ...   
           XYL            130.835                NaN                 1.0   
           YUM            134.870                NaN                 1.0   
           ZBH            103.550                NaN                 1.0   
           ZBRA           363.530                NaN                 1.0   
           ZTS            182.850                NaN                 1.0   

                   Next Day Close  Previous Day Close    Return  Volatility  \
Date       Ticker                                                             
2024-10-28 A                  NaN              130.19  0.010984    0.012264   
           AAPL               NaN              231.41  0.008988    0.013784   
           ABBV               NaN              187.85  0.010061    0.008103   
           ABNB               NaN              134.58  0.009214    0.014882   
           ABT                NaN              114.22 -0.000788    0.010339   
...                           ...                 ...       ...         ...   
           XYL                NaN              130.42  0.003182    0.009305   
           YUM                NaN              133.04  0.013755    0.006897   
           ZBH                NaN              102.35  0.011725    0.010933   
           ZBRA               NaN              360.09  0.009553    0.009969   
           ZTS                NaN              180.01  0.015777    0.013254   

                         RSI      SMA_50     SMA_100     SMA_200  Upper Band  \
Date       Ticker                                                              
2024-10-28 A       21.425411  140.169080  136.640840  137.961560  151.502270   
           AAPL    63.955185  226.950400  222.175250  201.011900  237.876570   
           ABBV    45.739376  192.653440  183.300860  174.407040  196.414630   
           ABNB    57.457336  125.337400  133.320100  143.597980  140.080760   
           ABT     50.131912  114.222595  109.517050  109.521515  119.484985   
...                      ...         ...         ...         ...         ...   
           XYL     39.970257  133.478420  134.334370  130.949620  138.418910   
           YUM     50.767624  134.574110  133.394600  134.047180  138.692950   
           ZBH     56.671688  107.229390  107.883995  115.284256  107.086190   
           ZBRA    43.505260  356.281000  337.452000  311.584440  380.047030   
           ZTS     42.451588  189.078800  183.390370  179.182240  197.458200   

                   Lower Band   Support  Resistance Action  Year  Month  Day  
Date       Ticker                                                             
2024-10-28 A        128.73973  130.1900   148.24400    NaN  2024     10   28  
           AAPL     222.29343  216.3200   236.48000    NaN  2024     10   28  
           ABBV     185.96626  186.5400   197.77069    NaN  2024     10   28  
           ABNB     125.49424  114.2800   137.19000    NaN  2024     10   28  
           ABT      111.09392  110.2504   119.39000    NaN  2024     10   28  
...                       ...       ...         ...    ...   ...    ...  ...  
           XYL      129.50359  126.7100   137.53000    NaN  2024     10   28  
           YUM      130.99506  129.7100   139.92000    NaN  2024     10   28  
           ZBH      101.50781  101.7700   115.91237    NaN  2024     10   28  
           ZBRA     359.12097  320.7700   377.68000    NaN  2024     10   28  
           ZT

In [9]:
# Drop columns that are not needed for the model
select_features_to_drop = ["Action", "Next Day Close", "Previous Day Close", "Resistance", "Upper Band", "SMA_50", "SMA_200"]
log_select_to_drop = ["Action", "Next Day Close", "Previous Day Close", "SMA_50", "Resistance", "Upper Band", "SMA_200","Next Day Close", "Adjusted Close", "Support", "Lower Band", "SMA_100"]

select_target_to_drop = ["Today to Tomorrow"]

predict_data_clean = predict_data_raw.drop(columns=[*select_features_to_drop, *select_target_to_drop])
log_predict_data_clean = predict_data_raw.drop(columns=[*log_select_to_drop, *select_target_to_drop])

predict_data_clean

Adjusted Close  Yesterday to Today    Return  Volatility  \
Date       Ticker                                                             
2024-10-28 A              131.620                 1.0  0.010984    0.012264   
           AAPL           233.490                 1.0  0.008988    0.013784   
           ABBV           189.740                 1.0  0.010061    0.008103   
           ABNB           135.820                 1.0  0.009214    0.014882   
           ABT            114.130                -1.0 -0.000788    0.010339   
...                           ...                 ...       ...         ...   
           XYL            130.835                 1.0  0.003182    0.009305   
           YUM            134.870                 1.0  0.013755    0.006897   
           ZBH            103.550                 1.0  0.011725    0.010933   
           ZBRA           363.530                 1.0  0.009553    0.009969   
           ZTS            182.850                 1.0  0.015777    0.013254   

                         RSI     SMA_100  Lower Band   Support  Year  Month  \
Date       Ticker                                                             
2024-10-28 A       21.425411  136.640840   128.73973  130.1900  2024     10   
           AAPL    63.955185  222.175250   222.29343  216.3200  2024     10   
           ABBV    45.739376  183.300860   185.96626  186.5400  2024     10   
           ABNB    57.457336  133.320100   125.49424  114.2800  2024     10   
           ABT     50.131912  109.517050   111.09392  110.2504  2024     10   
...                      ...         ...         ...       ...   ...    ...   
           XYL     39.970257  134.334370   129.50359  126.7100  2024     10   
           YUM     50.767624  133.394600   130.99506  129.7100  2024     10   
           ZBH     56.671688  107.883995   101.50781  101.7700  2024     10   
           ZBRA    43.505260  337.452000   359.12097  320.7700  2024     10   
           ZTS     42.451588  183.390370   181.45679  180.0100  2024     10   

                   Day  
Date       Ticker       
2024-10-28 A        28  
           AAPL     28  
           ABBV     28  
           ABNB     28  
           ABT      28  
...                ...  
           XYL      28  
           YUM      28  
           ZBH      28  
           ZBRA     28  
           ZTS      28  

[501 rows x 11 columns]

In [10]:
import joblib

file_name = "X_scaler.pkl"
file_path = f"../../models/christian's_models/{file_name}"

X_scaler = joblib.load(file_path)
print(f"Loaded {file_name} from {file_path}")

Loaded X_scaler.pkl from ../../models/christian's_models/X_scaler.pkl


In [11]:
file_name = "X_scaler_log.pkl"
file_path = f"../../models/christian's_models/{file_name}"

X_scaler_log = joblib.load(file_path)
print(f"Loaded {file_name} from {file_path}")

Loaded X_scaler_log.pkl from ../../models/christian's_models/X_scaler_log.pkl


In [12]:
X_scaled = X_scaler.transform(predict_data_clean)

X_scaled[:1]

array([[ 0.21845648,  0.95644196,  0.4568274 , -0.49818646, -1.89770035,
         0.26598292,  0.24739888,  0.28358385,  1.63653275,  1.03295107,
         1.40292582]])

In [13]:
X_scaled_log = X_scaler_log.transform(log_predict_data_clean)

X_scaled_log[:1]

array([[ 0.95644196,  0.4568274 , -0.49818646, -1.89770035,  1.63653275,
         1.03295107,  1.40292582]])

In [15]:
import pickle

model_names = [
    "clf_XGB_v2.pkl",
    "random_forest_classifier_v2.pkl",
    "logistic_regression_v1.pkl"
]

models = {}

for model_name in model_names:
    model_path = f"../../models/christian's_models/{model_name}"
    with open(model_path, "rb") as file:
        model = pickle.load(file)
        model_name = model_name.replace(".pkl", "")
        models[model_name] = model
        print(f"Loaded {model_name} from {model_path}")


Loaded clf_XGB_v2 from ../../models/christian's_models/clf_XGB_v2.pkl
Loaded random_forest_classifier_v2 from ../../models/christian's_models/random_forest_classifier_v2.pkl
Loaded logistic_regression_v1 from ../../models/christian's_models/logistic_regression_v1.pkl


In [16]:
y_XGB = models["clf_XGB_v2"].predict(X_scaled)
y_RFC = models["random_forest_classifier_v2"].predict(X_scaled)
y_LR = models["logistic_regression_v1"].predict(X_scaled_log)

# Change values where `0` is to `-1`
y_XGB[y_XGB == 0] = -1
y_RFC[y_RFC == 0] = -1
y_LR[y_LR == 0] = -1

y_XGB[:5], y_RFC[:5], y_LR[:5]

(array([1, 1, 1, 1, 1]), array([1, 1, 1, 1, 1]), array([1., 1., 1., 1., 1.]))

In [17]:
todays_data_clean = todays_data[["Date", "Ticker", "Adjusted Close", "Today to Tomorrow"]].copy()

todays_data_clean

,Date,Ticker,Adjusted Close,Today to Tomorrow
0,2024-10-28,A,131.620,NaN
1,2024-10-28,AAPL,233.490,NaN
2,2024-10-28,ABBV,189.740,NaN
3,2024-10-28,ABNB,135.820,NaN
4,2024-10-28,ABT,114.130,NaN
...,...,...,...,...
496,2024-10-28,XYL,130.835,NaN
497,2024-10-28,YUM,134.870,NaN
498,2024-10-28,ZBH,103.550,NaN
499,2024-10-28,ZBRA,363.530,NaN


In [18]:
todays_data_predict = todays_data_clean.copy()

# Change to integer
todays_data_predict["XGB Today to Tomorrow"] = y_XGB
todays_data_predict["RanFC Today to Tomorrow"] = y_RFC
todays_data_predict["Log_R Today to Tomorrow"] = y_LR.astype(int)


print(todays_data_predict["XGB Today to Tomorrow"].unique())
print(todays_data_predict["RanFC Today to Tomorrow"].unique())
print(todays_data_predict["Log_R Today to Tomorrow"].unique())
todays_data_predict.head(50)


[ 1 -1]
[ 1 -1]
[ 1 -1]


,Date,Ticker,Adjusted Close,Today to Tomorrow,XGB Today to Tomorrow,RanFC Today to Tomorrow,Log_R Today to Tomorrow
0,2024-10-28,A,131.62,NaN,1,1,1
1,2024-10-28,AAPL,233.49,NaN,1,1,1
2,2024-10-28,ABBV,189.74,NaN,1,1,1
3,2024-10-28,ABNB,135.82,NaN,1,1,1
4,2024-10-28,ABT,114.13,NaN,1,1,1
5,2024-10-28,ACGL,105.65,NaN,1,1,1
6,2024-10-28,ACN,361.45,NaN,1,1,1
7,2024-10-28,ADBE,481.17,NaN,1,1,1
8,2024-10-28,ADI,230.14,NaN,1,1,1
9,2024-10-28,ADM,57.24,NaN,1,1,1


In [19]:
file_name = "todays_data_predict"
file_path = f"../../data/raw_data/{file_name}"

save_data(todays_data_predict, file_path)

╔═══════════════════════════════════════════════════════════════╗
║File `todays_data_predict.zip` already exists. Overwriting file.║
╚═══════════════════════════════════════════════════════════════╝
╔═══════════════════════════════════════════════════════════════╗
║      File saved and zipped as `todays_data_predict.zip`       ║
╚═══════════════════════════════════════════════════════════════╝
